In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import re

In [2]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1011,tt1440129,209000000,303025485,Battleship,Taylor Kitsch|Alexander SkarsgÃ¥rd|Rihanna|Bro...,Peter Berg,The Battle for Earth Begins at Sea,"When mankind beams a radio signal into space, ...",131,Thriller|Action|Adventure|Science Fiction,Universal Pictures|Hasbro|Bluegrass Films|Film 44,4/11/2012,5.5,2012
1656,tt0329691,40000000,46546197,Torque,Martin Henderson|Ice Cube|Monet Mazur|Adam Sco...,Joseph Kahn,Circuit of Fire,Biker Cary Ford is framed by an old rival and ...,84,Action,Village Roadshow Pictures|Original Film|Warner...,1/16/2004,4.8,2004
132,tt1877832,250000000,746000000,X-Men: Days of Future Past,Hugh Jackman|James McAvoy|Patrick Stewart|Mich...,Bryan Singer,"To save the future, they must alter the past",The ultimate X-Men ensemble fights a war for t...,131,Action|Adventure|Fantasy|Science Fiction,Twentieth Century Fox Film Corporation|Donners...,5/15/2014,7.6,2014
416,tt1053424,32000000,18409891,Repo Men,Jude Law|Forest Whitaker|Alice Braga|Liev Schr...,Miguel Sapochnik,"For a price, any organ in your body can be rep...","In the future, medical technology has advanced...",111,Action|Science Fiction|Thriller|Crime,Universal Pictures|Stuber Productions|Dentsu|R...,3/18/2010,6.1,2010
994,tt1764651,100000000,312573423,The Expendables 2,Sylvester Stallone|Jason Statham|Dolph Lundgre...,Simon West,Back for War.,Mr. Church reunites the Expendables for what s...,103,Action|Adventure|Thriller,Nu Image Films|Millennium Films,8/8/2012,6.0,2012


In [3]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [4]:
answers = {} # создадим словарь для ответов

# тут другие ваши предобработки колонок например:

#the time given in the dataset is in string format.
#So we need to change this in datetime format
# ...
data_work = data.copy()
profit = data['revenue'] - data_work['budget']
data_work.insert(3, 'profit', profit)
data_work['release_date'] = pd.to_datetime(data_work['release_date'], format = '%m/%d/%Y')

   
#Функция для определения перечня встречающихся переменных
def compound_value (data,column):
    value = list(data[column])
    result = []
    for i in range(len(value)):
        if '|' in value[i]:
            name = value[i].split('|')
            result = result + name
        else: result.append(value[i])
    return result

#Функция для определения кол-ва переменных
def quantity_value (values):
    result = Counter()
    for word in values:
        result[word] += 1
    result = list(result.items())
    result.sort(key = lambda row:row[1], reverse = True)
    return result

list_month = ['January', 'February', 
             'March', 'April', 'May', 
             'June', 'July', 'August', 
             'September', 'October', 'November' , 
             'December']
list_season = ['Winter', 'Winter', 
             'Spring', 'Spring', 'Spring', 
             'Summer', 'Summer', 'Summer', 
             'Autumn', 'Autumn', 'Autumn' , 
             'Winter']

In [5]:
#добавление столбца 'season'
release_date_data = data_work['release_date']
season = []
for i in range(len(release_date_data)):
    season.append(list_season[release_date_data[i].month - 1])
data_work.insert(13, 'season', season)

In [6]:
# Добавление столбца с количеством слов в описании
tagline_data = data['tagline']
quantity_word_data = []
for i in range(len(tagline_data)):
    quantity_word = 0
    for word in re.split(r'\s+', tagline_data[i]):
        quantity_word += 1
    quantity_word_data.append(quantity_word)
data_work.insert(8, 'quantity_word', quantity_word_data)

In [7]:
#функция определения уникальных значений
def unique_value (column):
    result = []
    for name in column:
        if name not in result:
            result.append(name)
    return result

In [8]:
data_work.head(2)

,imdb_id,budget,revenue,profit,original_title,cast,director,tagline,quantity_word,overview,runtime,genres,production_companies,release_date,season,vote_average,release_year
0,tt0369610,150000000,1513528810,1363528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,Colin Trevorrow,The park is open.,4,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,2015-06-09,Summer,6.5,2015
1,tt1392190,150000000,378436354,228436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,George Miller,What a Lovely Day.,4,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,2015-05-13,Spring,7.1,2015


In [9]:
data_work.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   imdb_id               1889 non-null   object        
 1   budget                1889 non-null   int64         
 2   revenue               1889 non-null   int64         
 3   profit                1889 non-null   int64         
 4   original_title        1889 non-null   object        
 5   cast                  1889 non-null   object        
 6   director              1889 non-null   object        
 7   tagline               1889 non-null   object        
 8   quantity_word         1889 non-null   int64         
 9   overview              1889 non-null   object        
 10  runtime               1889 non-null   int64         
 11  genres                1889 non-null   object        
 12  production_companies  1889 non-null   object        
 13  release_date      

In [10]:
data_work.describe()

,budget,revenue,profit,quantity_word,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,1.010545e+08,7.044468,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,1.830733e+08,3.662992,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,-1.657101e+08,1.000000,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,5.000000e+06,4.000000,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,4.400041e+07,6.000000,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,1.201842e+08,9.000000,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,2.544506e+09,49.000000,214.000000,8.100000,2015.000000


# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [11]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
answers['1'] = '2. Spider-Man 3 (tt0413300)'
# запишите свой вариант ответа
answers['1'] = '...'
# если ответили верно, можете добавить комментарий со значком "+"

In [12]:
# тут пишем ваш код для решения данного вопроса:
index = data[data['budget'] == data['budget'].max()].index
answers['1'] = data.loc[index[0], 'original_title'] + '(' + data.loc[index[0], 'imdb_id'] + ')'
print(answers['1'])

Pirates of the Caribbean: On Stranger Tides(tt1298650)


ВАРИАНТ 2

In [13]:
# можно добавлять разные варианты решения

# 2. Какой из фильмов самый длительный (в минутах)?

In [14]:
# думаю логику работы с этим словарем вы уже поняли, 
# по этому не буду больше его дублировать
answers['2'] = '...'

In [15]:
index = data[data['runtime'] == data['runtime'].max()].index
answers['2'] = data.loc[index[0], 'original_title'] + '(' + data.loc[index[0], 'imdb_id'] + ')'
print(answers['2'])

Gods and Generals(tt0279111)


# 3. Какой из фильмов самый короткий (в минутах)?





In [16]:
index = data[data['runtime'] == data['runtime'].min()].index
answers['3'] = data.loc[index[0], 'original_title']  + '(' + data.loc[index[0], 'imdb_id'] + ')'
print(answers['3'])

Winnie the Pooh(tt1449283)


# 4. Какова средняя длительность фильмов?


In [17]:
answers['4'] = data['runtime'].mean()
print(answers['4'])

109.6585494970884


# 5. Каково медианное значение длительности фильмов? 

In [18]:
answers['5'] = data['runtime'].median()
print(answers['5'])

107.0


# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [19]:
# лучше код получения столбца profit вынести в Предобработку что в начале
index = data_work[data_work['profit'] == data_work['profit'].max()].index
answers['6'] = data_work.loc[index[0], 'original_title'] + '(' + data.loc[index[0], 'imdb_id'] + ')'
print(answers['6'])

Avatar(tt0499549)


# 7. Какой фильм самый убыточный? 

In [20]:
index = data_work[data_work['profit'] == data_work['profit'].min()].index
answers['7'] = data_work.loc[index[0], 'original_title']  + '(' + data.loc[index[0], 'imdb_id'] + ')'
print(answers['7'])

The Lone Ranger(tt1210819)


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [21]:
answers['8'] = data_work[data_work['profit'] > 0]['original_title'].count()
print(answers['8'])

1478


# 9. Какой фильм оказался самым кассовым в 2008 году?

In [22]:
film_2008 = data_work[data_work['release_year'] == 2008]
index = film_2008[film_2008['profit'] == film_2008['profit'].max()].index
answers['9'] = film_2008.loc[index[0], 'original_title']  + '(' + data.loc[index[0], 'imdb_id'] + ')'
print(answers['9'])

The Dark Knight(tt0468569)


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [23]:
film_2012_2014 = data_work[(data_work['release_year'] >= 2012) & (data_work['release_year'] <= 2014)]
index = film_2012_2014[film_2012_2014['profit'] == film_2012_2014['profit'].min()].index
answers['10'] = film_2012_2014.loc[index[0], 'original_title']  + '(' + data.loc[index[0], 'imdb_id'] + ')'
print(answers['10'])

The Lone Ranger(tt1210819)


# 11. Какого жанра фильмов больше всего?

In [24]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале
quantity_genres = quantity_value(compound_value(data_work, 'genres')) 
answers['11'] = quantity_genres[0]
print(answers['11'])

('Drama', 782)


ВАРИАНТ 2

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [25]:
film_profit = data_work[data_work['profit'] > 0]
quantity_genres_profit = quantity_value(compound_value(film_profit, 'genres')) 
answers['12'] = quantity_genres_profit[0]
print(answers['12'])

('Drama', 560)


# 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [26]:
director_revenue = data_work.groupby('director')['revenue'].sum().sort_values(ascending = False)
answers['13'] = director_revenue.index[0]
print(answers['13'])

Peter Jackson


# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [27]:
director_film_action = data_work[data_work['genres'].str.match('Action')]['director'].value_counts()
answers['14'] = director_film_action.index[0]
print(answers['14'])

Antoine Fuqua


# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [28]:
film_2012 = data_work[data_work['release_year'] == 2012]
index = film_2012[film_2012['revenue'] == film_2012['revenue'].max()]['cast'].index
answers['15'] = list(film_2012.loc[index, 'cast'])
display(answers['15'])

['Robert Downey Jr.|Chris Evans|Mark Ruffalo|Chris Hemsworth|Scarlett Johansson']

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [29]:
film_high_budget = data_work[data_work['budget'] > data_work['budget'].quantile(0.75)]
quantity_actor = quantity_value(compound_value(film_high_budget, 'cast')) 
answers['16'] = quantity_actor[0]
print(answers['16'])

('Adam Sandler', 16)


# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [30]:
film_cage = data_work[data_work['cast'].str.contains('Nicolas Cage')]
quantity_cage = quantity_value(compound_value(film_cage, 'genres')) 
answers['17'] = quantity_cage[0]
print(answers['17'])

('Action', 17)


# 18. Самый убыточный фильм от Paramount Pictures

In [31]:
film_paramount = data_work[data_work['production_companies'].str.contains('Paramount Pictures')]
index = film_paramount[film_paramount['profit'] == film_paramount['profit'].min()]['original_title'].index
answers['18'] = film_paramount.loc[index[0], 'original_title']  + '(' + data.loc[index[0], 'imdb_id'] + ')'
print(answers['18'])

K-19: The Widowmaker(tt0267626)


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [32]:
year_sum_revenue = data_work.groupby('release_year')['revenue'].sum().sort_values(ascending = False)
answers['19'] = year_sum_revenue.index[0]
print(answers['19'])

2015


# 20. Какой самый прибыльный год для студии Warner Bros?

In [33]:
film_warner = data_work[data_work['production_companies'].str.contains('Warner Bros')]
year_sum_profit_warner = film_warner.groupby('release_year')['profit'].sum().sort_values(ascending = False)
answers['20'] = year_sum_profit_warner.index[0]
print(answers['20'])

2014


# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [34]:
film_for_month = data_work.groupby(pd.DatetimeIndex(data_work['release_date']).month)['original_title'].count().\
sort_values(ascending = False)
answers['21'] = list_month[film_for_month.index[0]-1]
print(answers['21'])

September


# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [35]:
answers['22'] = film_for_month[6] + film_for_month[7] + film_for_month[8]
print(answers['22'])

450


# 23. Для какого режиссера зима – самое продуктивное время года? 

In [36]:
film_winter = data_work[data_work['season'] == 'Winter']
director_max_winter = film_winter['director'].value_counts()
answers['23'] = director_max_winter.index[0]
print(answers['23'])

Peter Jackson


# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [37]:
original_title_data = data_work['original_title']
length_title_data = []
index_max_length = 0
for i in range(len(original_title_data)):
    length_title_data.append(len(original_title_data[i]))
for i in range(len(length_title_data)):
    if length_title_data[i] == max(length_title_data):
        index_max_length = i
index_max_length
answers['24'] = data_work.loc[index_max_length, 'production_companies']
print(answers['24'])

Twentieth Century Fox Film Corporation|Four By Two Productions


# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [38]:
list_company = unique_value(compound_value (data_work, 'production_companies'))
company_quantity_word = []
for row in list_company:
    quantity_word = 0
    quantity_film = 0
    for i in range(len(data_work['production_companies'])):
        if row in data_work['production_companies'][i]:
            quantity_word += data_work['quantity_word'][i]
            quantity_film += 1
    company_quantity_word.append([row, quantity_word, quantity_film])
company_tagline = pd.DataFrame(data = company_quantity_word, columns = ['production_companies', 'sum_word', 'quantity_film'])

In [39]:
company_tagline.head()

,production_companies,sum_word,quantity_film
0,Universal Studios,37,6
1,Amblin Entertainment,174,23
2,Legendary Pictures,199,34
3,Fuji Television Network,13,3
4,Dentsu,84,13


In [40]:
company_tagline['mean_word'] = company_tagline['sum_word'] / company_tagline['quantity_film']
company_tagline.head()

,production_companies,sum_word,quantity_film,mean_word
0,Universal Studios,37,6,6.166667
1,Amblin Entertainment,174,23,7.565217
2,Legendary Pictures,199,34,5.852941
3,Fuji Television Network,13,3,4.333333
4,Dentsu,84,13,6.461538


In [91]:
index = company_tagline[company_tagline['mean_word'] == company_tagline['mean_word'].max()].index
answers['25'] = company_tagline.loc[index[0], 'production_companies']
print (answers['25'])

Beverly Detroit


In [93]:
company_tagline['mean_word'].sort_values(ascending = False)

1111    49.000000
1444    24.000000
1112    23.333333
467     21.000000
230     19.666667
          ...    
594      1.000000
1360     1.000000
1538     1.000000
1374     1.000000
1375     1.000000
Name: mean_word, Length: 1771, dtype: float64

In [109]:
company_tagline[company_tagline['production_companies'] == 'Total Entertainment']

,production_companies,sum_word,quantity_film,mean_word
975,Total Entertainment,5,1,5.0


# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [48]:
limit_min = data_work['vote_average'].max() - 10/100
limit_max = data_work['vote_average'].max()
data_vote_average = data_work.query('@limit_min <= vote_average <= @limit_max')
answers['26'] = list(data_vote_average['original_title'])
print(answers['26'])

['Inside Out', 'Room', 'Interstellar', 'The Imitation Game', 'The Dark Knight']


# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


ВАРИАНТ 2

In [84]:
list_actor = unique_value(compound_value(data_work, 'cast'))
#создаем список актеров
list_two_actor = []
actor_1 = ''
actor_2 = ''
for i in range(len(list_actor)-1):
    actor_1 = list_actor[i]
    for j in range((i+1),len(list_actor)):
        actor_2 = list_actor[j]
        list_two_actor.append([actor_1, actor_2])

In [85]:
result_list_actor = []
for i in range(len(list_two_actor)):
    quantity_two_actor = 0
    two_actor = list_two_actor[i][0] +'&'+ list_two_actor[i][1]
    #определяем строки DataFrame с заданными актерами
    index_for_first = list(data_work[data_work['cast'].str.contains(list_two_actor[i][0])].index)
    index_for_second = list(data_work[data_work['cast'].str.contains(list_two_actor[i][1])].index)
    #Сравнение списков строк для определения участия двух актеров в фильме
    for j in range(len(index_for_first)):
        for k in range(len(index_for_second)):
            if index_for_first[j] == index_for_second[k]:
                quantity_two_actor += 1
                result_list_actor.append([two_actor, quantity_two_actor])    
list_two_actor_data = pd.DataFrame(data = result_list_actor, columns = ['name_actor', 'quantity_cast'])

In [89]:
answers['27'] = list_two_actor_data[list_two_actor_data['quantity_cast'] == list_two_actor_data['quantity_cast'].max()]
print(answers['27'])                              

                          name_actor  quantity_cast
7813    Daniel Radcliffe&Emma Watson              8
7821   Daniel Radcliffe&Rupert Grint              8
10693       Emma Watson&Rupert Grint              8


# Submission

In [110]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides(tt1298650)',
 '2': 'Gods and Generals(tt0279111)',
 '3': 'Winnie the Pooh(tt1449283)',
 '4': 109.6585494970884,
 '5': 107.0,
 '6': 'Avatar(tt0499549)',
 '7': 'The Lone Ranger(tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight(tt0468569)',
 '10': 'The Lone Ranger(tt1210819)',
 '11': ('Drama', 782),
 '12': ('Drama', 560),
 '13': 'Peter Jackson',
 '14': 'Antoine Fuqua',
 '15': ['Robert Downey Jr.|Chris Evans|Mark Ruffalo|Chris Hemsworth|Scarlett Johansson'],
 '16': ('Adam Sandler', 16),
 '17': ('Action', 17),
 '18': 'K-19: The Widowmaker(tt0267626)',
 '19': 2015,
 '20': 2014,
 '21': 'September',
 '22': 450,
 '23': 'Peter Jackson',
 '24': 'Twentieth Century Fox Film Corporation|Four By Two Productions',
 '25': 'Beverly Detroit',
 '26': ['Inside Out',
  'Room',
  'Interstellar',
  'The Imitation Game',
  'The Dark Knight'],
 '27':                           name_actor  quantity_cast
 7813    Daniel Radcliffe&Emma Watson              8
 7821   D

In [ ]:
# и убедиться что ни чего не пропустил)
len(answers)